# Choose an Optimal Credit Card Stack, Optimized for Your Specific Expenses. 

Name: Hunter Worssam \
Date: 25 August 2025

## Notes
This current iteration requires the user to populate their desired spend categories for each card, as well as the % cashback or point value for each category. This model will not take into account special offers outside of spend categories, such as Amex's cash back on Disney+, or Chase's hotel credits. It also does not yet account for sign-up bonuses & waived fee promotions.

### Install required packages

In [4]:
!pip install pulp pandas

In [5]:
import pulp
import pandas as pd
from collections import defaultdict
from typing import Dict, List, Union, Tuple, Set, Optional

## Define algorithm

In [7]:
def optimize_credit_cards(
    cards: Dict[str, Dict[str, float]],
    spending: Dict[str, float],
    annual_fees: Dict[str, float],
    allow_unassigned: bool = True,     # enable a 0% fallback so bad choices are not enforced
    max_cards: Union[int, None] = None, # cap number of cards you “carry”; None = no cap
    keep_nonnegative: bool = True,     # keep overall objective ≥ 0 (can set False if you want raw optimum)
) -> Union[Tuple[Dict[str, str], float, Set[str]], str]:
    """
    Optimizes credit card usage to maximize rewards minus annual fees.

    Returns:
        chosen_cards (dict): category -> chosen card (or 'NoCard' when unassigned and allowed)
        total_reward (float): net value (rewards - fees)
        unique_cards (set): unique set of real cards chosen (excludes 'NoCard')
    """

    # Optional 0% fallback so categories aren't forced into a bad card.
    WORK_cards = dict(cards) 
    WORK_fees = dict(annual_fees)
    if allow_unassigned:
        WORK_cards["NoCard"] = {cat: 0.0 for cat in spending} 
        WORK_fees["NoCard"] = 0.0

    # Initialize the MILP
    prob = pulp.LpProblem("Maximize_Credit_Card_Rewards", pulp.LpMaximize)

    # Decision variables split into:
    # - x[card][category] ∈ {0,1} (use card for that category)
    # - y[card] ∈ {0,1} (hold the card at all)
    x = {
        card: {category: pulp.LpVariable(f"x_{card}_{category}", 0, 1, pulp.LpBinary)
               for category in spending}
        for card in WORK_cards
    }
    y = {card: pulp.LpVariable(f"y_{card}", 0, 1, pulp.LpBinary) for card in WORK_cards}  # ADDED

    #  If card lacks a category key, fall back to a 'Base' key if present, else 0.0. 
    # This avoids KeyErrors and unrealistic zeros.
    def earn_rate(card: str, category: str) -> float:
        card_table = WORK_cards[card]
        return card_table.get(category, card_table.get("Base", 0.0))

    # Objective = sum(rewards) - sum(card_fees counted ONCE via y)
    rewards = pulp.lpSum(
        x[card][cat] * spending[cat] * earn_rate(card, cat)
        for card in WORK_cards for cat in spending
    )
    fees = pulp.lpSum(y[card] * WORK_fees[card] for card in WORK_cards)
    total_reward_expr = rewards - fees
    prob += total_reward_expr

    # Linking constraint so you can't use a card in a category unless you "hold" it.
    for card in WORK_cards:
        for cat in spending:
            prob += x[card][cat] <= y[card]

    # Keep the "exactly one" assignment per category, but because of the 'NoCard' option with 0%, we are not forced
    # to pick a bad paid card just to satisfy the equality.
    for cat in spending:
        prob += pulp.lpSum(x[card][cat] for card in WORK_cards) == 1 

    # Optional cap on number of cards to carry
    if max_cards is not None:
        # Don't count 'NoCard' against the cap
        prob += pulp.lpSum(y[c] for c in WORK_cards if c != "NoCard") <= int(max_cards)

    # Positivity constraint is optional.
    # With 'NoCard' present, the model can always hit 0 by picking nothing.
    if keep_nonnegative:
        prob += total_reward_expr >= 0 

    # Solve
    prob.solve()

    # Handle optimality / feasibility
    if pulp.LpStatus[prob.status] == "Optimal":
        chosen_cards = {}
        for cat in spending:
            # identify the unique chosen card (ties shouldn't happen under binary + equality)
            chosen = None
            for card in WORK_cards:
                if pulp.value(x[card][cat]) == 1:
                    chosen = card
                    break
            # Safety fallback
            chosen_cards[cat] = chosen if chosen is not None else ("NoCard" if allow_unassigned else next(iter(WORK_cards)))

        total_reward = float(pulp.value(prob.objective))

        # Unique real cards (exclude 'NoCard' if present)
        unique_cards = {c for c in WORK_cards if c != "NoCard" and pulp.value(y[c]) == 1}

        return chosen_cards, total_reward, unique_cards

    # If infeasible (e.g., keep_nonnegative=True plus weird inputs), match intended return style
    return "No positive reward solution found", 0, set()

# Function for a Clean PnL Table Output

In [9]:
def make_pnl_table(
    chosen_cards: Dict[str, str],
    spending: Dict[str, float],
    cards: Dict[str, Dict[str, float]],
    annual_fees: Dict[str, float],
    unique_cards: Set[str],
    include_nocard: bool = False,   # show a "NoCard" row if present
    sort_by: str = "Net Value ($)", # or "Rewards ($)"
) -> pd.DataFrame:
    """
    Build a per-card P&L table consistent with your solver:
    - Rewards are summed for the categories assigned to each card.
    - Annual fee is counted ONCE per card (cards in `unique_cards`).
    - Net Value = Rewards - Annual Fee
    - Break-even Spend = fee / blended reward rate (if defined)
    """

    # Helper for earn rate with a safe Base fallback
    def earn_rate(card: str, cat: str) -> float:
        tbl = cards.get(card, {})
        return tbl.get(cat, tbl.get("Base", 0.0))

    # Gather categories used by each card
    cats_by_card = defaultdict(list)
    for cat, card in chosen_cards.items():
        if (card == "NoCard") and not include_nocard:
            continue
        cats_by_card[card].append(cat)

    rows = []
    all_cards = set(cats_by_card.keys()) | set(unique_cards)
    # Optionally include NoCard (0 fee, 0 rewards)
    if include_nocard and "NoCard" in cats_by_card:
        all_cards.add("NoCard")

    for card in sorted(all_cards):
        # Assigned spend & rewards from chosen categories
        assigned_cats = cats_by_card.get(card, [])
        spend = sum(spending[c] for c in assigned_cats)
        rewards = sum(spending[c] * earn_rate(card, c) for c in assigned_cats)

        # Fee counted once per card actually carried
        fee = float(annual_fees.get(card, 0.0)) if (card in unique_cards) else 0.0
        if card == "NoCard":
            fee = 0.0  # ensure

        net = rewards - fee

        # Blended reward rate (cash per $) for break-even calc
        blended_rate = (rewards / spend) if spend > 0 else 0.0
        be_spend = (fee / blended_rate) if blended_rate > 0 else float("inf")

        rows.append({
            "Card": card,
            "Categories Used": ", ".join(sorted(assigned_cats)) if assigned_cats else "—",
            "Assigned Spend ($)": round(spend, 2),
            "Rewards ($)": round(rewards, 2),
            "Annual Fee ($)": round(fee, 2),
            "Net Value ($)": round(net, 2),
            "Break-Even Spend ($)": ("—" if be_spend == float("inf") else round(be_spend, 2)),
        })

    df = pd.DataFrame(rows)
    if not df.empty:
        df = df.sort_values([sort_by, "Rewards ($)"], ascending=[False, False]).reset_index(drop=True)

        # Add a totals row (excluding NoCard)
        mask_real = df["Card"] != "NoCard"
        totals = {
            "Card": "TOTAL",
            "Categories Used": "—",
            "Assigned Spend ($)": round(df.loc[mask_real, "Assigned Spend ($)"].sum(), 2),
            "Rewards ($)": round(df.loc[mask_real, "Rewards ($)"].sum(), 2),
            "Annual Fee ($)": round(df.loc[mask_real, "Annual Fee ($)"].sum(), 2),
            "Net Value ($)": round(df.loc[mask_real, "Net Value ($)"].sum(), 2),
            "Break-Even Spend ($)": "—",
        }
        df = pd.concat([df, pd.DataFrame([totals])], ignore_index=True)

    return df

## Example Usuage

In [11]:
cards = {
    'Amex_Blue': {'Groceries': 0.06, 'Gas': 0.03, 'Dining': 0.01, 'Travel': 0.01, 'Entertainment': 0.01, 'Education': 0.01, 'Utilities': 0.01, 'Costco': 0.01},
    'Chase_Saph': {'Groceries': 0.01, 'Gas': 0.01, 'Dining': 0.03, 'Travel': 0.02, 'Entertainment': 0.02, 'Education': 0.01, 'Utilities': 0.01, 'Costco': 0.01},
    'Amex_Gold': {'Groceries': 0.04, 'Gas': 0.01, 'Dining': 0.04, 'Travel': 0.025, 'Entertainment': 0.01, 'Education': 0.01, 'Utilities': 0.01, 'Costco': 0.01},
    'Venture': {'Groceries': 0.02, 'Gas': 0.02, 'Dining': 0.02, 'Travel': 0.02, 'Entertainment': 0.02, 'Education': 0.02, 'Utilities': 0.02, 'Costco': 0.02},
    'Savor': {'Groceries': 0.03, 'Gas': 0.01, 'Dining': 0.03, 'Travel': 0.01, 'Entertainment': 0.03, 'Education': 0.01, 'Utilities': 0.01, 'Costco': 0.01},
    'Costco': {'Groceries': 0.01, 'Gas': 0.04, 'Dining': 0.03, 'Travel': 0.01, 'Entertainment': 0.01, 'Education': 0.01, 'Utilities': 0.01, 'Costco': 0.02},
    'Chase_Freedom': {'Groceries': 0.015, 'Gas': 0.015, 'Dining': 0.03, 'Travel': 0.015, 'Entertainment': 0.015, 'Education': 0.015, 'Utilities': 0.015, 'Costco': 0.015}
}

annual_spending = {
    'Groceries': 4000,
    'Gas': 1200,
    'Dining': 6500,
    'Travel': 2500,
    'Entertainment': 2800,
    'Education': 10000,
    'Utilities': 1700,
    'Costco': 1400,
}

annual_fees = {
    'Amex_Blue': 95,
    'Chase_Saph': 95,
    'Amex_Gold': 325,
    'Venture': 95,
    'Savor': 0,
    'Costco': 0,
    'Chase_Freedom': 0
}

In [12]:
chosen_cards, total_reward, unique_cards = optimize_credit_cards(cards, annual_spending, annual_fees, max_cards=4)

print("Chosen Cards by Category:", chosen_cards)
print("Total Reward:", round(total_reward, 2))
print("Unique Cards Selected:", unique_cards)
print("Number of Cards to Carry:", len(unique_cards))
print("")

pnl = make_pnl_table(
    chosen_cards=chosen_cards,
    spending=annual_spending,
    cards=cards,
    annual_fees=annual_fees,
    unique_cards=unique_cards,
    include_nocard=False 
)
print(pnl.to_string(index=False))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/dv/mzhtb9w54p32nmxwmm3vt7340000gn/T/8cae2be4fb2d474e9f1147e98c20d2c1-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/dv/mzhtb9w54p32nmxwmm3vt7340000gn/T/8cae2be4fb2d474e9f1147e98c20d2c1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 79 COLUMNS
At line 543 RHS
At line 618 BOUNDS
At line 691 ENDATA
Problem MODEL has 74 rows, 72 columns and 259 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 713 - 0.00 seconds
Cgl0004I processed model has 66 rows, 71 columns (71 integer (71 of which binary)) and 243 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0038I Initial state - 0 integers unsatisfied sum - 6.66134e-16
Cbc0038I Solution found of -713
Cbc0038I Be